
# Team Matteo and Dias Two legends
## This notebook was late in 5 minutes, score 0.13812
### just to show that we worked and not just changed the similarity to cosine
Members: Dias Khalniyasov khalniyasov.1954228@studenti.uniroma1.it

Matteo Di Mauro dimauro.1954323@studenti.uniroma1.it

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  import tensorflow.compat.v2 as tf
except Exception:
  pass
tf.enable_v2_behavior()

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#from __future__ import absolute_import, division, print_function, unicode_literals
import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
from tqdm.auto import tqdm
import csv
import pandas as pd

# Reload Data and Preprocess

In [2]:
# Download caption annotation files
annotation_folder = '/annotations/'
if not os.path.exists(os.path.abspath('.') + annotation_folder):
  annotation_zip = tf.keras.utils.get_file('captions.zip',
                                            cache_subdir=os.path.abspath('.'),
                                            origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                            extract = True)
  annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'
  os.remove(annotation_zip)

# Download image files
image_folder = '/train2014/'
if not os.path.exists(os.path.abspath('.') + image_folder):
  image_zip = tf.keras.utils.get_file('train2014.zip',
                                      cache_subdir=os.path.abspath('.'),
                                      origin = 'http://images.cocodataset.org/zips/train2014.zip',
                                      extract = True)
  PATH = os.path.dirname(image_zip) + image_folder
  os.remove(image_zip)
else:
  PATH = os.path.abspath('.') + image_folder

13510582272/13510573713 [==============================] - 795s 0us/step


In [3]:
annotation_file = './annotations/captions_train2014.json'

PATH = './train2014'

In [4]:
# Read the json file
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

# Store captions and image names in vectors
all_captions = []
all_img_name_vector = []

for annot in annotations['annotations']:
    caption = '<start> ' + annot['caption'] + ' <end>'
    image_id = annot['image_id']
    full_coco_image_path = os.path.join(PATH, 'COCO_train2014_' + '%012d.jpg' % (image_id))

    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)

# Shuffle captions and image_names together
# Set a random state
train_captions, img_name_vector = shuffle(all_captions,
                                          all_img_name_vector,
                                          random_state=1)

# Select the first 30000 captions from the shuffled set
num_examples = 30000
train_captions = train_captions[:num_examples]
img_name_vector = img_name_vector[:num_examples]

In [5]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [6]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

87924736/87910968 [==============================] - 2s 0us/step


In [7]:
# Get unique images
encode_train = sorted(set(img_name_vector))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in tqdm(image_dataset):
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

  0%|          | 0/1622 [00:00<?, ?it/s]

In [8]:
# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)


# Choose the top 5000 words from the vocabulary
top_k = 5000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
#train_seqs = tokenizer.texts_to_sequences(train_captions)

tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

# Calculates the max_length, which is used to store the attention weights
max_length = calc_max_length(train_seqs)

In [9]:
# Create training and validation sets using an 80-20 split
img_name_train, img_name_val, cap_train, cap_val = train_test_split(img_name_vector,
                                                                    cap_vector,
                                                                    test_size=0.0333,
                                                                    random_state=0)

# NEW: Work with Caption to Got Most Similar Images with Your Query

## Reload previous Results
Here you can start to test your model. We provide you a baseline model that use Jaccard similarity in order to compare 2 captions.
This is a first approach and you have to improve it! 

Loading the Data and "all_captions.csv" file you don't have to train again all image captioning model. 

# Mount Drive

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [78]:
all_captions = pd.read_csv("/content/gdrive/My Drive/all_captions.csv", sep=',') 

real_captions = all_captions['true_caption']
pred_captions = all_captions['pred_caption']

## Use Glove pretrained as similarity criterion

In [13]:
import warnings
import cv2

In [98]:
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('wordnet')

# Removing text from stopwords, lowering and 

def cleaning(data):
    no_punct=re.sub('[^a-zA-Z]', ' ',data)
    lower=str.lower(no_punct).split()
    
    stop_words=set(stopwords.words('english'))
    clean = [word for word in lower if not word in stop_words]
    
    return ( " ".join(clean))  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
ls

annotations/  gdrive/  sample_data/  train2014/


In [17]:
# Removing dataset, as we need more storage to load pretrained model
import shutil

shutil.rmtree('train2014')

In [18]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-09-04 13:01:11--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-09-04 13:01:12--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-09-04 13:01:12--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [19]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [20]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.300d.txt'
word2vec_output_file = 'word2vec.txt'
# The first step is to convert the GloVe file format to the word2vec file format. 
# The only difference is the addition of a small header line. This can be done by calling the 
# glove2word2vec() function.

glove2word2vec(glove_input_file, word2vec_output_file)

(400000, 300)

In [21]:
# If there is an error on reading the header
#!pip install pycocotools==2.0.0

     |████████████████████████████████| 1.5 MB 3.4 MB/s 
  Created wheel for pycocotools: filename=pycocotools-2.0.0-cp37-cp37m-linux_x86_64.whl size=264068 sha256=fec3ece3e194214d761b01d7d674e1978e669b0507f6b7f8cda17bf6ed302913
  Stored in directory: /root/.cache/pip/wheels/4d/50/dc/e1f07e9eb5678a0ee21bc091220f1f3806ba8e48ef3f2083cb
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2


In [22]:
#Loading vectors
embeddings_index = {}
with open('glove.6B.300d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [23]:
from gensim.models import KeyedVectors
glove_input_file = 'glove.6B.300d.txt'
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [33]:
#Setting up vocabulary of the model
w2v_vocab = set(model.vocab)

print("Loaded {} words in vocabulary".format(len(w2v_vocab)))

Loaded 400000 words in vocabulary


In [80]:
#Cleaning predicted sentences
prediction = all_captions['pred_caption']
for i in range(len(prediction)):
  prediction[i]=cleaning(prediction[i])

In [81]:
def count_distance(target_sentence, w2v_vocab = w2v_vocab,prediction=prediction):
  target_sentence = cleaning(target_sentence)
  # use n_similarity to compute a cosine similarity (should be reasonably robust)
  sentences_similarity = np.zeros(len(prediction))

  target_sentence_words = [w for w in target_sentence.split() if w in w2v_vocab]
  for idx, sentence in enumerate(prediction):
      sentence_words = [w for w in sentence.split() if w in w2v_vocab]
      sim = model.n_similarity(target_sentence_words, sentence_words)
      sentences_similarity[idx] = sim
  result = list(zip(sentences_similarity, prediction))
  
  print("Target:", target_sentence)
  return result

  # Show the target phrase and the most-similar target phrases

## Create File with Your Submission Results

In [95]:
def create_submission_file(top_k, img_name_val, real_captions, pred_captions):

    with open('./submission_glove.csv', 'w') as file:
        writer = csv.writer(file)
        writer.writerow(["caption", "image_list"])

        for idx in tqdm(range(len(img_name_val))):
            result = count_distance(real_captions[idx])
            scores_id = sorted(range(len(result)), key=lambda k: result[k],reverse=True)
            result = result.sort(key=lambda item:item[0], reverse=True)
            writer.writerow([' '.join(real_captions[idx]), ' '.join(list(map(lambda x: str(x), scores_id[:top_k])))])

In [96]:
create_submission_file(len(img_name_val), img_name_val, real_captions, prediction)

  0%|          | 0/1000 [00:00<?, ?it/s]

Target: white canoe body water front red bench
Target: man kiteboarding waves ocean
Target: team playing baseball baseball diamond
Target: group players reach frisbee
Target: dog following two people horses beach
Target: cycle seen going road
Target: tiny dirty little bathroom
Target: people standing beach flying kite
Target: veggie wrap sitting plate covered gravy
Target: person holding pair scissors orange handles
Target: picture residential street sign trees
Target: old courtyard people browsing around outside
Target: table small apartment set one person
Target: image small stove oven
Target: cat laying front mirror
Target: couple people polar bear hammock
Target: plate food table small knife next
Target: people walk past display old suitcases
Target: couple giraffes standing rock wall
Target: group tourists surprised curious giraffes
Target: close slice cake
Target: two people walking snow towards buildings
Target: two black white clocks tan building white blue bus
Target: bike lea

In [97]:
from google.colab import files
files.download('submission_glove.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## View Some Results

### Show Qualitative Results for a Choosen Index

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize

stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])

In [ ]:
def show_image(image_fname, new_figure=True):
  if new_figure:
    plt.figure()
  np_img = cv2.imread(image_fname)
  np_img = cv2.cvtColor(np_img, cv2.COLOR_BGR2RGB)
  plt.imshow(np_img) 

def show_qualitative_results(idx1, top_k=20):

    b_score_res = get_similar_result_jaccard(idx1, real_captions, pred_captions)

    print("Real capt:", ' '.join(real_captions[idx1]))
    print("Pred capt:", ' '.join(pred_captions[idx1]))
    sentence1 = [w for w in real_captions[idx1] if not w in stop_words]
    sentence2 = [w for w in pred_captions[idx1] if not w in stop_words]
    ss = jaccard_similarity(sentence1, sentence2)
    print("Score with True Predicted caption:", ss)
    print()

    show_image(img_name_val[idx1], new_figure=False)
    plt.grid(False)
    plt.ioff()
    plt.axis('off')


    fig = plt.figure(figsize=(10, 7))

    for idx2, (idx, sim_val) in enumerate(b_score_res[:20]):
        print(idx, sim_val, ' '.join(pred_captions[idx]))
        plt.subplot(4, 5, idx2+1)
        show_image(img_name_val[idx], new_figure=False)
        plt.grid(False)
        plt.ioff()
        plt.axis('off')
        plt.title('{}'.format(idx2+1))

In [ ]:
show_qualitative_results(idx1 = 0)

### Show Distribution of Right Prediction

In [ ]:
all_idx = []
top_k = 1000

for ref_idx in tqdm(range(len(img_name_val))):
    s_score_res = get_similar_result_jaccard(ref_idx, real_captions, pred_captions)
    list_res = list(map(lambda x: x[0], s_score_res[:top_k]))
    index = list_res.index(ref_idx)
    all_idx.append(index)

n, bins, patches = plt.hist(all_idx, bins=1000)
plt.xlabel('top K')
plt.ylabel('Frequency')

plt.show()